Data from: https://datasets.imdbws.com



In [2]:
import pandas as pd
import re

Import datasets

In [3]:
ratings = pd.read_csv("title_ratings.tsv", sep="\t")
titles = pd.read_csv("title_basics.tsv", sep="\t")
akas = pd.read_csv("title_akas.tsv", sep = "\t")

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Combine the data

In [4]:
df = titles.join(ratings.set_index('tconst'), on='tconst')
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1967.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,263.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1803.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,179.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2603.0


Get number of types of data

In [5]:
df["titleType"].value_counts()

tvEpisode       7376874
short            921241
movie            640226
video            272536
tvSeries         240799
tvMovie          140846
tvMiniSeries      47853
tvSpecial         40956
videoGame         33816
tvShort           10001
tvPilot               1
Name: titleType, dtype: int64

Drop entries that have no votes, or no average rating

In [6]:
df = df.dropna(subset=['averageRating', 'numVotes'])

In [7]:
df["titleType"].value_counts()

tvEpisode       631394
movie           289368
short           145973
tvSeries         86276
tvMovie          50240
video            49066
tvMiniSeries     14695
videoGame        14435
tvSpecial        11008
tvShort           2138
Name: titleType, dtype: int64

We only want the alternate titles from the akas dataframe. We can remove the other columns, drop the duplicates and group all the alternate titles in a single list for each id.

In [8]:
akas = akas.drop(["ordering", "region", "language", "types", "attributes", "isOriginalTitle"], axis = 1)
akas = akas.drop_duplicates()
akas = akas.groupby('titleId').agg({'title': lambda x: list(x)})
akas

,title
titleId,
tt0000001,"[Карменсіта, Carmencita, Carmencita - spanyol ..."
tt0000002,"[Le clown et ses chiens, A bohóc és kutyái, De..."
tt0000003,"[Sarmanul Pierrot, Szegény Pierrot, 哀れなピエロ, Бі..."
tt0000004,"[Un bon bock, Ein gutes Glas Bier, Un ţap de b..."
tt0000005,"[Blacksmith Scene, The Blacksmith's Forge, Bla..."
...,...
tt9916846,"[Épisode #3.18, Folge #3.18, エピソード #3.18, Epis..."
tt9916848,"[Épisode #3.17, Folge #3.17, エピソード #3.17, Epis..."
tt9916850,"[Episódio #3.19, एपिसोड #3.19, Épisode #3.19, ..."


Isolate the video games and combine with the akas dataframe

In [9]:
video_games = df[df["titleType"]=="videoGame"]
video_games = video_games.join(akas, on='tconst').rename(columns = {"title": "akas"})
video_games

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,akas
82552,tt0084376,videoGame,"MysteryDisc: Murder, Anyone?","MysteryDisc: Murder, Anyone?",0,1982,\N,\N,"Adventure,Crime,Mystery",6.1,37.0,"[MysteryDisc: Murder, Anyone?]"
102667,tt0105000,videoGame,Night Trap,Night Trap,0,1992,\N,\N,"Adventure,Horror,Mystery",6.2,376.0,"[Scene of the Crime, Night Trap, ナイト トラップ]"
107391,tt0109865,videoGame,Gabriel Knight: Sins of the Fathers,Gabriel Knight: Sins of the Fathers,0,1993,\N,\N,"Adventure,Drama,Horror",9.1,706.0,[Gabriel Knight: Sins of the Fathers]
107783,tt0110267,videoGame,King's Quest VII: The Princeless Bride,King's Quest VII: The Princeless Bride,0,1994,\N,\N,"Adventure,Fantasy",7.6,206.0,"[King's Quest VII: The Princeless Bride, King'..."
108416,tt0110909,videoGame,Psychic Detective,Psychic Detective,0,1995,\N,\N,"Adventure,Fantasy,Mystery",8.5,50.0,[Psychic Detective]
...,...,...,...,...,...,...,...,...,...,...,...,...
9712195,tt9888864,videoGame,Infernal,Infernal,0,2007,\N,\N,Action,6.0,17.0,"[Infernal, Infernal: Hell's Vengeance, Diaboli..."
9713877,tt9892552,videoGame,Spiral Splatter,Spiral Splatter,0,2017,\N,\N,Action,2.0,10.0,[Spiral Splatter]
9714234,tt9893348,videoGame,Alien: Covenant In Utero VR Experience,Alien: Covenant In Utero VR Experience,0,2017,\N,\N,Sci-Fi,6.2,17.0,[Alien: Covenant In Utero VR Experience]
9714437,tt9893804,videoGame,The Lego Movie 2 Videogame,The Lego Movie 2 Videogame,0,2019,\N,\N,"Adventure,Animation,Family",5.5,156.0,"[The Lego Movie 2 Videogame, The Lego Movie 2,..."


Save the video games file to a csv

In [29]:
video_games.to_csv("imdb_games_db.csv")

# UNUSED/TEST CODE FROM BELOW

In [16]:
principals = pd.read_csv("name_basics.tsv", sep="\t")

In [17]:
principals.sample()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
9278014,nm6026807,László Kiss,\N,\N,actor,"tt3132658,tt2981532"


In [15]:
principals[principals["tconst"]=="tt0206583"]

,tconst,directors,writers
198298,tt0206583,nm10566918,"nm0277730,nm0004170"


In [ ]:
titles[titles["primaryTitle"]=="LEGO Star Wars"]

In [ ]:
with_winnie = titles['primaryTitle'].str.contains('[^a-z]lego[^a-z]|$lego[^a-z]', na=False, flags=re.IGNORECASE, regex=True)
with_movie_type = titles['titleType']=='movie'

titles[with_winnie & with_movie_type]

In [ ]:
temp = titles[titles["primaryTitle"].str.contains('Winnie', na=False, flags=re.IGNORECASE, regex=True)]
temp[temp["titleType"]=="movie"].head(65)

In [ ]:
titles[titles["primaryTitle"].str.contains('Stranger Things', na=False, flags=re.IGNORECASE, regex=True)]